In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader
import random
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", device)

cpu와 cuda 중 다음 기기로 학습함: cuda


In [3]:
# train_word=['Repeat is the best medicine for memory asd zxc','you are the best dreamer for drug']

In [4]:
# words = [i.split() for i in train_word]

In [5]:
df = pd.read_csv('C:\\Users\\pc\\Downloads\\IMDB Dataset.csv\\IMDB Dataset.csv')

In [6]:
df = df.review
df= df[:2500]

In [7]:
word = [i.split(". ") for i in df]

In [8]:
word = sum(word,[])

In [9]:
import string
words =[]
for voca in word:
    if len(voca.split())< 20:
        text = [i.lower() for i in voca.split()]
        text = [i.replace("<br","") for i in text]
        text = [''.join(j for j in i if j not in string.punctuation) for i in text]
        text = [s for s in text if s]
        if len(text) == 5:
            words.append(text)

In [10]:
len(words)

549

In [11]:
word2index ={}
for voca in words:
    for word in voca:
        if word not in word2index.keys():
            word2index[word] = len(word2index)+1

In [12]:
idx2char = {word2index[i] : i for i in word2index}
idx2char[0]=''

In [13]:
max_len = max([len(i) for i in words])
max_len

5

In [14]:
decode = lambda y: [idx2char.get(x) for x in y]

In [15]:
sentences= [[word2index[j] for j in i] for i in words]
# sentences = np.array(sentences)

In [16]:
sequences=[]
for voca in sentences:
    for i in range(1,len(voca)):
        sequence = voca[:i+1]
        sequences.append(sequence)

In [17]:
train_vector=[]

for lines in sequences:
    if len(lines) < max_len: # 현재 샘플이 정해준 길이보다 짧으면
        a=[0] * (max_len - len(lines)) # 나머지는 전부 'pad' 토큰으로 채운다.
        a+=lines
        train_vector.append(a)
    else:
        train_vector.append(lines)

In [18]:
len(train_vector)

2196

In [59]:
train_vector = np.array(train_vector)
x=train_vector[:,:-1]
y=train_vector[:,1:]
# y = np.eye(voca_size, dtype='uint8')[y]
x=torch.LongTensor(x).to(device)
y=torch.LongTensor(y).to(device)

In [60]:
num_words=x.size(0)

In [61]:
x.shape,y.shape

(torch.Size([2196, 4]), torch.Size([2196, 4]))

In [62]:
voca_size = len(word2index)+1
embeding_dim = 128
hidden_size = 128

In [63]:

# batch_size = 3342
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.emd = nn.Embedding(voca_size,embeding_dim)
        self.rnn = nn.RNN(embeding_dim,hidden_size,num_layers=3)
        self.fc = nn.Linear(hidden_size,voca_size)
    
    def forward(self,x):
        # print(x.shape)
        x = self.emd(x)
        # print(x.shape)
        x,hidden = self.rnn(x)
        x,hidden = self.rnn(x)
        # print(x.shape)
        x = self.fc(x)
        # print(x.shape)
        # print(x.view(-1,x.size(2)).shape)
        return x.view(-1,x.size(2))

In [65]:

rnn = RNN().to(device)


criterion = nn.CrossEntropyLoss() #1
optimizer = optim.Adam(rnn.parameters(),lr = 0.001)


for epoch in range(1001):
    optimizer.zero_grad()
#     print("predicted string : ",end="")

    outputs = rnn(x)
    loss = criterion(outputs,y.view(-1))
    _, pred = outputs.max(1)
    pred = pred.cpu().detach().numpy().reshape(num_words,-1)
    result_str = [" ".join([idx2char[c] for c in id]) for id in pred]
    if epoch % 10 == 0:
        print("epoch: %d, loss: %1.3f" % (epoch+1,loss.data))
    loss.backward()
    optimizer.step()
    

epoch: 1, loss: 6.956
epoch: 11, loss: 5.591
epoch: 21, loss: 5.137
epoch: 31, loss: 5.030
epoch: 41, loss: 4.854
epoch: 51, loss: 4.705
epoch: 61, loss: 4.602
epoch: 71, loss: 4.520
epoch: 81, loss: 4.449
epoch: 91, loss: 4.388
epoch: 101, loss: 4.333
epoch: 111, loss: 4.280
epoch: 121, loss: 4.227
epoch: 131, loss: 4.172
epoch: 141, loss: 4.114
epoch: 151, loss: 4.049
epoch: 161, loss: 3.973
epoch: 171, loss: 3.887
epoch: 181, loss: 3.821
epoch: 191, loss: 3.776
epoch: 201, loss: 3.647
epoch: 211, loss: 3.561
epoch: 221, loss: 3.478
epoch: 231, loss: 3.392
epoch: 241, loss: 3.310
epoch: 251, loss: 3.239
epoch: 261, loss: 3.148
epoch: 271, loss: 3.087
epoch: 281, loss: 3.017
epoch: 291, loss: 2.926
epoch: 301, loss: 2.876
epoch: 311, loss: 2.781
epoch: 321, loss: 2.707
epoch: 331, loss: 2.644
epoch: 341, loss: 2.611
epoch: 351, loss: 2.509
epoch: 361, loss: 2.445
epoch: 371, loss: 2.377
epoch: 381, loss: 2.328
epoch: 391, loss: 2.269
epoch: 401, loss: 2.203
epoch: 411, loss: 2.206
epo

In [29]:
["".join(idx2char[i]) for i in sentences[3]]

['i', 'hated', 'them', 'so', 'much']

In [26]:
result_str

['   for',
 '  for keitel',
 'i for keitel obsessives',
 'for keitel obsessives only',
 '  this is',
 ' this is one',
 'this is one of',
 'is one of them',
 '  this movie',
 ' this movie was',
 'this movie was so',
 'movie was so frustrating',
 '  this hated',
 ' this hated them',
 'this hated them so',
 'hated them so much',
 '  horrible and',
 ' horrible and beautiful',
 'it and beautiful at',
 'and beautiful at once',
 '  just dont',
 ' i dont watch',
 'i dont watch the',
 'dont watch the movie',
 '  i give',
 ' i give this',
 'i give this a',
 'give this a 1',
 '  honestly this',
 ' honestly this short',
 'wood this short film',
 'this short film sucks',
 '  its not',
 ' its not just',
 'its all just about',
 'the just about entertainment',
 '  this crude',
 ' i crude yet',
 'i crude yet awakeningly',
 'crude yet awakeningly fresh',
 '  you just',
 ' you just cant',
 'it just cant hate',
 'just cant hate it',
 '  stephen hawkings',
 ' stephen hawkings is',
 'stephen hawkings is a',